In [1]:
#import required libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline 
import seaborn as sns

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

In [3]:
# impoting the dataset
data = pd.read_csv('labeled_datasetFinal.csv')
data.head()

,recipe_name,number_of_ingerdients,number_of_preparation_words,cooking_time,ingredients,preparation:,label
0,Hasanpaşa köftesi tarifi,16,150,70,"['500 gram kıyma', '1 soğan', '1 yumurta', '3 ...","Püreyi yapmakla başlayın, patateslerin kabukla...",hard
1,Kuymak tarifi,5,69,30,"['2 yemek kaşığı mısır unu', '2 yemek kaşığı t...",Tavaya tereyağını alın ve orta ateşte yakmadan...,easy
2,Tavuklu İspanyol salatası tarifi,7,50,30,"['1 adet tavuk göğüs eti', '10 adet cherry dom...",Yeşillikleri yıkayıp süzün. Tavuk göğsünü july...,hard
3,Parmak bonfile tarifi,8,71,45,"['3 adet bonfile', '1 tatlı kaşığı tereyağı', ...","Ispanakları ayıklayın, yıkayın ve kaynayan suy...",hard
4,Şuale tarifi,6,52,45,"['1 su bardağı su', '100 gram tereyağı', '1 ye...","Su, tereyağı, toz şeker ve tuzu bir tencereye ...",hard


In [4]:
#split data
X= data.iloc[:,1:4]
y=  data.iloc[:,6:7]
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test= train_test_split(X,y,test_size =0.2,random_state=5)


In [5]:
# Use sklearn utility to convert label strings to numbered index
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

/home/momen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/momen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/momen/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(["easy","hard"])
print(transfomed_label)

[[0]
 [1]]


In [7]:
#let's start creating the model
import keras
import sklearn.model_selection
from keras.models import Sequential 
from keras.layers import Dense
import sys


Using TensorFlow backend.


In [8]:
#Define model
model = keras.models.Sequential()
def create_model(model):
    model.add(Dense( units=10, kernel_initializer='uniform', activation='relu',input_dim = 3))
    #second layer.
    model.add(Dense( units=10, kernel_initializer='uniform', activation='relu'))
    model.add(Dense( units=5, kernel_initializer='uniform', activation='relu'))
    # add the output layer    
    model.add(Dense( units=1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs =100, batch_size =5,verbose=0)
    scores = model.evaluate(x_train, y_train)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return model
model=create_model(model)


3782/3782 [==============================] - 0s 43us/step
acc: 79.88%


In [9]:
# 16	150	70
x_train.head()

,number_of_ingerdients,number_of_preparation_words,cooking_time
3782,5,40,45
3075,9,53,30
3896,7,98,45
1343,11,73,30
4088,10,164,45


In [10]:
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 211
Trainable params: 211
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
y_pred = model.predict(x_test)
y_pred= y_pred>0.5


In [20]:
# test is with one recipe
from numpy import array
data = array([11, 22,  55])
data = data.reshape((3,1))
data.shape

y_pred= model.predict(np.array([[4, 22,  55]]))
y_pred= y_pred>0.5
y_pred

array([[False]])

In [26]:

import pickle

# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = 'MLP.pkl'
# Open the file to save as pkl file
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
pickle.dump(model, decision_tree_model_pkl)
# Close the pickle instances
decision_tree_model_pkl.close()

In [27]:
model_pkl = open('MLP.pkl', 'rb')
saved_model = pickle.load(model_pkl)

In [28]:
y_pred= saved_model.predict(np.array([[2, 2,  55]]))
y_pred= y_pred>0.5
y_pred

array([[False]])